
# In-class Activity: Feedforward Neural Network (From Scratch) + Framework Comparison (PyTorch)

This notebook mirrors the homework expectations:

- **2D input**
- **Two hidden layers** with **sigmoid** activations
- **2-class output** with **softmax**
- **No bias terms**
- **SGD training** (single-example updates)
- **Track and plot training loss**
- **Report final weights**
_toggle into PyTorch and compare with precision / recall / F1 on the test set._

> Data files are expected at: `data/week04/train.csv` and `data/week04/test.csv`.



## What you should learn today

By the end of this activity, you should be able to:

1. Implement a forward pass with clear **matrix shapes**
2. Implement backprop for a small network **without relying on a framework**
3. Run **SGD** and track loss over time
4. Diagnose common training pathologies (e.g., predicting only one class)
5. Compare a scratch implementation to a **PyTorch baseline** under the same constraints

### Suggested workflow during class
- Run the notebook once top-to-bottom.
- Then experiment with **learning rate**, **epochs**, **hidden sizes**, and **initialization seed**.


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

from sklearn.metrics import precision_recall_fscore_support, confusion_matrix



## 1) Load data

We load CSVs from `data/week04/`.  
If the files are not present, we generate a simple synthetic 2D dataset so that the notebook still runs.


In [ ]:

def load_or_make_data(train_path="data/week04/train.csv", test_path="data/week04/test.csv", seed=0):
    train_path = Path(train_path)
    test_path = Path(test_path)

    if train_path.exists() and test_path.exists():
        train_df = pd.read_csv(train_path)
        test_df  = pd.read_csv(test_path)

        # Assumption: last column is the label
        X_train = train_df.iloc[:, :-1].to_numpy(dtype=np.float64)
        y_train = train_df.iloc[:,  -1].to_numpy()
        X_test  = test_df.iloc[:,  :-1].to_numpy(dtype=np.float64)
        y_test  = test_df.iloc[:,   -1].to_numpy()

        # If labels are one-hot, convert to indices
        if y_train.ndim > 1 and y_train.shape[1] > 1:
            y_train = np.argmax(y_train, axis=1)
            y_test  = np.argmax(y_test, axis=1)

        y_train = y_train.astype(int)
        y_test  = y_test.astype(int)

        return X_train, y_train, X_test, y_test, "loaded_csv"

    # ---- fallback synthetic dataset ----
    rng = np.random.default_rng(seed)
    n_train, n_test = 400, 200

    mu0 = np.array([-1.0, -1.0])
    mu1 = np.array([ 1.0,  1.0])
    cov = np.array([[0.5, 0.0],
                    [0.0, 0.5]])

    X0 = rng.multivariate_normal(mu0, cov, size=n_train//2)
    X1 = rng.multivariate_normal(mu1, cov, size=n_train//2)
    X_train = np.vstack([X0, X1])
    y_train = np.hstack([np.zeros(n_train//2, dtype=int),
                         np.ones(n_train//2, dtype=int)])

    X0t = rng.multivariate_normal(mu0, cov, size=n_test//2)
    X1t = rng.multivariate_normal(mu1, cov, size=n_test//2)
    X_test = np.vstack([X0t, X1t])
    y_test = np.hstack([np.zeros(n_test//2, dtype=int),
                        np.ones(n_test//2, dtype=int)])

    # shuffle
    idx = rng.permutation(n_train)
    X_train, y_train = X_train[idx], y_train[idx]
    idt = rng.permutation(n_test)
    X_test, y_test = X_test[idt], y_test[idt]

    return X_train, y_train, X_test, y_test, "synthetic"

X_train, y_train, X_test, y_test, data_mode = load_or_make_data()
print("data_mode:", data_mode)
print("X_train:", X_train.shape, "y_train:", y_train.shape, "classes:", np.unique(y_train))
print("X_test :", X_test.shape,  "y_test :", y_test.shape,  "classes:", np.unique(y_test))


In [ ]:

# Quick visualization (assumes 2D features, as in the homework)
plt.figure()
plt.scatter(X_train[:,0], X_train[:,1], c=y_train, s=18)
plt.title("Training data (colored by class)")
plt.xlabel("x1"); plt.ylabel("x2")
plt.show()



## 2) Activation + Softmax (numerically stable)

Key ideas:
- Clamp sigmoid input to prevent overflow in `exp()`
- Subtract `max(logits)` before softmax so that `exp()` is stable

We also use one-hot encoding for the loss and for the backprop convenience.


In [ ]:

def sigmoid(z):
    z = np.clip(z, -50, 50)
    return 1.0 / (1.0 + np.exp(-z))

def softmax(z):
    z = z - np.max(z, axis=-1, keepdims=True)
    expz = np.exp(z)
    return expz / np.sum(expz, axis=-1, keepdims=True)

def one_hot(y, num_classes):
    y = np.asarray(y, dtype=int)
    out = np.zeros((y.shape[0], num_classes), dtype=np.float64)
    out[np.arange(y.shape[0]), y] = 1.0
    return out



## 3) Cross-entropy loss

For a 2-class softmax output, we use:

$
\mathcal{L} = -\sum_{c} y_c \log(\hat{p}_c)
$

Where `y` is one-hot and `p` are softmax probabilities.


In [ ]:

def cross_entropy_loss(probs, y_onehot, eps=1e-12):
    probs = np.clip(probs, eps, 1.0)
    return -np.sum(y_onehot * np.log(probs), axis=1)  # per-sample



## 4) From-scratch network (no biases)

Architecture:
- Input (2) → Hidden1 (`h1`, sigmoid) → Hidden2 (`h2`, sigmoid) → Output (2, softmax)

**No bias terms** means every layer is only a matrix multiplication: `X @ W`.


In [ ]:

class TwoHiddenNN:
    def __init__(self, d_in=2, h1=3, h2=3, d_out=2, seed=0):
        rng = np.random.default_rng(seed)
        self.W1 = rng.normal(0, 0.2, size=(d_in, h1)).astype(np.float64)
        self.W2 = rng.normal(0, 0.2, size=(h1, h2)).astype(np.float64)
        self.W3 = rng.normal(0, 0.2, size=(h2, d_out)).astype(np.float64)

    def forward(self, X):
        Z1 = X @ self.W1
        A1 = sigmoid(Z1)

        Z2 = A1 @ self.W2
        A2 = sigmoid(Z2)

        Z3 = A2 @ self.W3
        P  = softmax(Z3)

        cache = {"X": X, "Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2, "Z3": Z3, "P": P}
        return P, cache

    def backward(self, cache, y_onehot):
        X  = cache["X"]
        A1 = cache["A1"]
        A2 = cache["A2"]
        P  = cache["P"]
        Z1 = cache["Z1"]
        Z2 = cache["Z2"]

        N = X.shape[0]

        # Softmax + CrossEntropy gives: dZ3 = (P - Y)/N
        dZ3 = (P - y_onehot) / N
        dW3 = A2.T @ dZ3

        dA2 = dZ3 @ self.W3.T
        s2 = sigmoid(Z2)
        dZ2 = dA2 * (s2 * (1.0 - s2))
        dW2 = A1.T @ dZ2

        dA1 = dZ2 @ self.W2.T
        s1 = sigmoid(Z1)
        dZ1 = dA1 * (s1 * (1.0 - s1))
        dW1 = X.T @ dZ1

        return {"dW1": dW1, "dW2": dW2, "dW3": dW3}

    def predict(self, X):
        P, _ = self.forward(X)
        return np.argmax(P, axis=1)

    def get_weights(self):
        return {"W1": self.W1.copy(), "W2": self.W2.copy(), "W3": self.W3.copy()}



## 5) SGD training loop (single-example updates)

We do **stochastic gradient descent** by:
1. Shuffling examples each epoch
2. For each example: forward → loss → backward → update weights

We log the loss **every step** so you can plot it.


In [ ]:
def train_sgd(model, X, y, lr=0.1, epochs=30, seed=0, shuffle=True):
    rng = np.random.default_rng(seed)
    losses = []

    K = len(np.unique(y))
    n = X.shape[0]

    for ep in range(epochs):
        idx = rng.permutation(n) if shuffle else np.arange(n)

        for i in idx:
            Xi = X[i:i+1]           # (1,2)
            yi = y[i:i+1]           # (1,)
            Yi = one_hot(yi, K)     # (1,2)

            P, cache = model.forward(Xi)
            loss = cross_entropy_loss(P, Yi).item()
            losses.append(loss)

            grads = model.backward(cache, Yi)

            model.W1 -= lr * grads["dW1"]
            model.W2 -= lr * grads["dW2"]
            model.W3 -= lr * grads["dW3"]

        # small progress report
        if (ep + 1) % max(1, epochs // 5) == 0:
            yhat = model.predict(X)
            acc = (yhat == y).mean()
            print(f"epoch {ep+1:3d}/{epochs} | last_step_loss={losses[-1]:.4f} | train_acc={acc:.3f}")

    return np.array(losses, dtype=np.float64)


### Try-it-now: adjust parameters

Start with the defaults, then experiment:

- `lr` (learning rate): try **0.01, 0.05, 0.1, 0.2**
- `epochs`: try **30, 60, 100**
- `h1/h2`: try **(3,3), (5,5), (10,10)**
- `seed`: try different seeds to see variance

Watch:
- loss curve shape
- training accuracy
- whether predictions collapse to a single class


In [ ]:

# ===== Experiment parameters =====
h1, h2 = 3, 3
lr = 0.1
epochs = 30
seed = 0

scratch_model = TwoHiddenNN(h1=h1, h2=h2, seed=seed)
losses = train_sgd(scratch_model, X_train, y_train, lr=lr, epochs=epochs, seed=seed)

plt.figure()
plt.plot(losses)
plt.title("From-scratch SGD training loss (per step)")
plt.xlabel("SGD step")
plt.ylabel("Cross-entropy loss")
plt.show()

final_weights_scratch = scratch_model.get_weights()
{k: v.shape for k, v in final_weights_scratch.items()}



## 6) Evaluation metrics + common warning explained

You might see something like:

```
UndefinedMetricWarning: Precision is ill-defined ... due to no predicted samples
```

This happens when the model predicts **only one class** on the test set.

### Why you might see Precision = Recall = F1 = 0.0

If the scratch model predicts **only one class**, then:
- Precision/recall for the *other* class is undefined.
- `sklearn` warns you and sets that metric to 0 unless you choose `zero_division`.

### What to try if this happens
- Increase epochs (e.g. **60–100**)
- Try a smaller learning rate (e.g. **0.05**)
- Increase hidden sizes (e.g. **5** or **10** neurons)
- Try different random seeds

We use **macro averaging** so both classes are considered equally.


In [ ]:

def eval_macro_metrics(y_true, y_pred):
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average="macro", zero_division=0
    )
    return prec, rec, f1

yhat_test_scratch = scratch_model.predict(X_test)
prec_s, rec_s, f1_s = eval_macro_metrics(y_test, yhat_test_scratch)

print("From-scratch metrics on test (macro avg):")
print(" precision:", prec_s)
print(" recall   :", rec_s)
print(" f1       :", f1_s)
print("confusion matrix:\n", confusion_matrix(y_test, yhat_test_scratch))

# Helpful diagnostic: how many predictions of each class?
vals, counts = np.unique(yhat_test_scratch, return_counts=True)
print("predicted class counts:", dict(zip(vals, counts)))



## 7) Framework comparison: PyTorch (same constraints)

We build the same architecture in PyTorch:
- 2 hidden sigmoid layers
- no bias terms (`bias=False`)
- 2 output logits

Then we train with SGD and evaluate precision/recall/F1 the same way.

> PyTorch’s `CrossEntropyLoss` expects **logits**, not probabilities, so we do **not** apply softmax in the forward pass.


In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim

# Keep everything in float64 to match NumPy more closely (optional)
torch.set_default_dtype(torch.float64)

class TorchTwoHidden(nn.Module):
    def __init__(self, d_in=2, h1=3, h2=3, d_out=2):
        super().__init__()
        self.fc1 = nn.Linear(d_in, h1, bias=False)
        self.fc2 = nn.Linear(h1, h2, bias=False)
        self.fc3 = nn.Linear(h2, d_out, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        logits = self.fc3(x)
        return logits

# Tensors
Xtr_t = torch.from_numpy(X_train)
ytr_t = torch.from_numpy(y_train.astype(np.int64))
Xte_t = torch.from_numpy(X_test)
yte_t = torch.from_numpy(y_test.astype(np.int64))

torch_model = TorchTwoHidden(h1=h1, h2=h2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(torch_model.parameters(), lr=lr)

torch_losses = []
n = X_train.shape[0]

for ep in range(epochs):
    perm = torch.randperm(n)
    for i in perm:
        xi = Xtr_t[i:i+1]
        yi = ytr_t[i:i+1]

        optimizer.zero_grad()
        logits = torch_model(xi)
        loss = criterion(logits, yi)
        loss.backward()
        optimizer.step()

        torch_losses.append(loss.item())

    if (ep + 1) % max(1, epochs // 5) == 0:
        with torch.no_grad():
            pred_train = torch.argmax(torch_model(Xtr_t), dim=1)
            acc = (pred_train == ytr_t).double().mean().item()
        print(f"epoch {ep+1:3d}/{epochs} | last_step_loss={torch_losses[-1]:.4f} | train_acc={acc:.3f}")

plt.figure()
plt.plot(torch_losses)
plt.title("PyTorch SGD training loss (per step)")
plt.xlabel("SGD step")
plt.ylabel("Cross-entropy loss")
plt.show()

with torch.no_grad():
    yhat_test_torch = torch.argmax(torch_model(Xte_t), dim=1).cpu().numpy()

prec_t, rec_t, f1_t = eval_macro_metrics(y_test, yhat_test_torch)

print("PyTorch metrics on test (macro avg):")
print(" precision:", prec_t)
print(" recall   :", rec_t)
print(" f1       :", f1_t)
print("confusion matrix:\n", confusion_matrix(y_test, yhat_test_torch))

vals, counts = np.unique(yhat_test_torch, return_counts=True)
print("predicted class counts:", dict(zip(vals, counts)))



## 8) Side-by-side comparison

This is the table you can paste into your homework write-up.


In [ ]:

results = pd.DataFrame([
    {"model": "from_scratch_numpy", "precision": prec_s, "recall": rec_s, "f1": f1_s},
    {"model": "pytorch_framework",  "precision": prec_t, "recall": rec_t, "f1": f1_t},
])
results



## 9) Final weights (scratch model)

The homework asks you to report the final weights.


In [ ]:

final_weights_scratch



## Question to Consider

If PyTorch does better than scratch, list two reasons besides “PyTorch is better.”
